# Working with Assistants Part 1
The Assistants API allows you to build AI assistants within your own applications. An Assistant has instructions and can leverage models, tools, and files to respond to user queries. The Assistants API currently supports three types of tools: Code Interpreter, File Search, and Function calling.

A typical integration of the Assistants API has the following flow:

1. Create an Assistant by defining its custom instructions and picking a model. If helpful, add files and enable tools like Code Interpreter, File Search, and Function calling.
2. Create a Thread when a user starts a conversation.
3. Add Messages to the Thread as the user asks questions.
4. Run the Assistant on the Thread to generate a response by calling the model and the tools.

## Calling Pre-Existing Assistants with Code
To use a preexisting Assistant we have to get the Assistant id first. We can do this through the interface or through code. Below we show how to do this with code.

In [ ]:
# Uncomment the below line and run this to make sure 
# you have the latest version of the openai package
# !pip install --upgrade openai

In [ ]:
# Import the OpenAI class from the openai module.
from openai import OpenAI

# Create an instance of the OpenAI class.
# NOTE: This assumes you have an OPENAI_API_KEY environment variable set.
client = OpenAI()

# Retrieve a list of AI assistants available under your account in descending order.
# The list is limited to the top 20 assistants.
my_assistants = client.beta.assistants.list(
    order="desc",
    limit=20  # Ensure limit is an integer, not a string
)

# Print the data attribute from the my_assistants object to display the assistants.
print(my_assistants.data)
print("\n\n")

# Iterate through each assistant in the list and print its name and ID.
for assistant in my_assistants.data:
    print(f"Name: {assistant.name}, ID: {assistant.id}")

print("\n\n")


Now we can retrieve the Assistant by the id number

In [ ]:
# Uncomment the below line and run this to make sure
# you have the pytz package installed which is used 
# for timezone conversion
# !pip install pytz

import datetime
import pytz

# Import the OpenAI class from the openai module.
from openai import OpenAI

# Create an instance of the OpenAI class.
client = OpenAI()

# Retrieve a specific assistant using its unique identifier.
my_assistant = client.beta.assistants.retrieve("asst_2Y9kU1ZtaqtHNbNm83s78CV1")

# Print the retrieved assistant to display its details.
print(my_assistant)
print("\n\n")
print(my_assistant.name)

# grab the created date/time in unix format 
# and convert it to a human readable format
unix_time = my_assistant.created_at

# Create a timezone object for Central Time
central_timezone = pytz.timezone('America/Chicago')

# Convert Unix time to a datetime object in UTC
utc_date = datetime.datetime.utcfromtimestamp(unix_time)

# Localize the UTC datetime object to Central Time
central_date = utc_date.replace(tzinfo=pytz.utc).astimezone(central_timezone)

# Format the datetime object to a readable string
formatted_date = central_date.strftime('%Y-%m-%d %H:%M:%S %Z')

print(formatted_date)



## Creating a Minimal Assistant
Let's begin by creating an Assistant through code. As a reminder, you can also create Assistants through the user interface at https://platform.openai.com/assistants as well. 


In [ ]:
# Import the OpenAI class from the openai module.
from openai import OpenAI

# Create an instance of the OpenAI class.
# NOTE: Make sure you have an environment variable, 
# called OPENAI_API_KEY set with your API key before running this code.
client = OpenAI()

# Create an assistant specifying only the required 'model' parameter.
# The 'model' is set to 'gpt-4-turbo'. All other parameters are optional and not specified here.
assistant = client.beta.assistants.create(
    model="gpt-4-turbo",
)

# The 'assistant' object now contains information about the newly created assistant.
# You could add code here to handle or display the 'assistant' object as needed, for example:
print(assistant)


In [ ]:
# Import the OpenAI class from the openai module.
from openai import OpenAI

# Create an instance of the OpenAI class.
# NOTE: This assumes you have an OPENAI_API_KEY environment variable set.
client = OpenAI()

# Retrieve a list of AI assistants available under your account in descending order.
# The list is limited to the top 20 assistants.
my_assistants = client.beta.assistants.list(
    order="desc",
    limit=20  # Ensure limit is an integer, not a string
)

# Print the data attribute from the my_assistants object to display the assistants.
print(my_assistants.data)
print("\n\n")

# Iterate through each assistant in the list and print its name and ID.
for assistant in my_assistants.data:
    print(f"Name: {assistant.name}, ID: {assistant.id}")

print("\n\n")


## Creating a Basic Assistant

Now let's create an Assistant with the following arguments:

**model**
(string)

*Required*

ID of the model to use. You can use the List models API to see all of your available models, or see our Model overview for descriptions of them.

---

**name**
(string or null)

*Optional*

The name of the assistant. The maximum length is 256 characters.

---

**description**
(string or null)

*Optional*

The description of the assistant. The maximum length is 512 characters.

---

**instructions**
(string or null)

*Optional*

The system instructions that the assistant uses. The maximum length is 256,000 characters.

In [ ]:
# Import the OpenAI class from the openai module.
from openai import OpenAI

# Create an instance of the OpenAI class.
client = OpenAI()

# Create an assistant with a basic configuration
assistant = client.beta.assistants.create(
    model="gpt-4-turbo",
    name="Seriously Cool Assistant",
    description="This is a seriously cool assistant",
    instructions="You are a helpful assistant."
)

# Print the details of the created assistant object to check the creation status and attributes.
print(assistant)
print("\n\n")


## Additional Arguments
**metadata**
(map)

*Optional*
Set of 16 key-value pairs that can be attached to an object. This can be useful for storing additional information about the object in a structured format. Keys can be a maximum of 64 characters long and values can be a maxium of 512 characters long.

---

**temperature**
(number or null)

*Optional*
(Defaults to 1)
What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.

---

**top_p**
(number or null)

*Optional*
(Defaults to 1)
An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.

---

**response_format**
(string or object)

*Optional*
(Defaults to "auto")
Specifies the format that the model must output. Compatible with GPT-4 Turbo and all GPT-3.5 Turbo models since gpt-3.5-turbo-1106.

Setting to { "type": "json_object" } enables JSON mode, which guarantees the message the model generates is valid JSON.

Important: when using JSON mode, you must also instruct the model to produce JSON yourself via a system or user message. Without this, the model may generate an unending stream of whitespace until the generation reaches the token limit, resulting in a long-running and seemingly "stuck" request. Also note that the message content may be partially cut off if finish_reason="length", which indicates the generation exceeded max_tokens or the conversation exceeded the max context length.

In [ ]:
# Import the OpenAI class from the openai module.
from openai import OpenAI

# Make sure you have the latest version of the OpenAI Python SDK installed.
# To update, run: pip install openai --upgrade

# Create an instance of the OpenAI class.
client = OpenAI()

# Create an assistant with a comprehensive set of arguments
assistant = client.beta.assistants.create(
    model="gpt-4-turbo",
    name="Extra Arguments Assistant",
    description="This assistant has extra arguments",
    instructions="You are a helpful assistant.",
    metadata={
        "key": "value",
        "IsAwesome": "True",
        "IsBeta": "False"
    },
    temperature=1,
    top_p=1,
    response_format="auto",
)

# Print the details of the created assistant to verify its properties.
print(assistant)
print("\n\n")


## Identifying Tools
**tools**
(array)

*Optional*
(Defaults to [])
A list of tool enabled on the assistant. There can be a maximum of 128 tools per assistant. Tools can be of types code_interpreter, file_search, or function.

---

**tool_resources**
(object or null)

*Optional*
A set of resources that are used by the assistant's tools. The resources are specific to the type of tool. For example, the code_interpreter tool requires a list of file IDs, while the file_search tool requires a list of vector store IDs.

In [ ]:
from openai import OpenAI

# Create an instance of the OpenAI class.
client = OpenAI()

# Create an assistant that uses code interpreter.
assistant = client.beta.assistants.create(
    instructions="You are a personal data analyst. When asked a question, write and run Python code to answer the question.",
    name="Code Interpreter Assistant",
    metadata={
        "key": "value",
        "is_awesome": "True",  # Lowercase dictionary keys
        "is_beta": "False"
    },
    temperature=1,
    top_p=1,
    response_format="auto",
    tools=[{"type": "code_interpreter"}],
    tool_resources=None,
    model="gpt-4-turbo",
)

# Print the details of the created assistant to check the properties.
print(assistant)
print("\n\n")

